# Let's get started! ⚡

In [1]:
## import Module

# Exploratory Data Analysis
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

# Data Preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin

# RF Training
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

# evaluation
from sklearn.metrics import accuracy_score

# Data Reading

In [2]:
# 데이터 읽어오기
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [ ]:
# 헤드 확인
df_train.head()

In [ ]:
# 데이터의 컬럼 수 확인
print(df_train.shape, df_test.shape)

In [ ]:
# 컬럼별 타입과 Null값 확인
print(df_train.info())
print()
print(df_test.info())

## Exploratory Data Analysis 📊🤔

In [ ]:
def eda_bar_plots(feature: str, target: str = 'Survived') -> None:
    """
    The given 'feature' is expected to be a categorical feature.
    Avoid calling this function with 'feature' == 'Age' or 'Fare'

    This function crates 3 bar plots to perform a visual
    Data Exploration analysis of the given 'feature'.
    """

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))

    # (1) the distribution of 'feature' in the Training data
    df1 = df_train[feature].value_counts(normalize=True)
    df1 = df1.sort_index().reset_index()
    # Now df1 has the columns: feature and 'proportion'
    sns.barplot(data=df1, x=feature, y='proportion', ax=ax1)
    # Set the title, the xlabel, and the ylabel.
    ax1.set(
        xlabel=feature,
        ylabel='% of each category',
        title=feature + ': % value counts [Training Data]'
    )
    # Plot a horizontal line at the proportion of a balanced
    # distribution of the categories in 'feature'
    ax1.axhline(
        y=1/len(df1), color='green', alpha=0.4, linestyle='--'
    )

    # (2) the distribution of 'feature' in the Test data
    df2 = df_test[feature].value_counts(normalize=True)
    df2 = df2.sort_index().reset_index()
    # Now df2 has the columns: feature and 'proportion'
    sns.barplot(data=df2, x=feature, y='proportion', ax=ax2)
    # Set the title, the xlabel, and the ylabel.
    ax2.set(
        xlabel=feature,
        ylabel='% of each category',
        title=feature + ': % value counts [Test Data]'
    )
    # Plot a horizontal line at the proportion of a balanced
    # distribution of the categories in 'feature'
    ax2.axhline(
        y=1/len(df2), color='green', alpha=0.4, linestyle='--'
    )

    # (3) the relationship between the 'target' and the
    # categories of 'feature' in the training data
    df3 = df_train.groupby(feature, as_index=False).agg(
        {target: 'mean'}
    ).sort_values(by=feature)
    sns.barplot(x=feature, y=target, data=df3, ax=ax3)
    ax3.set(
        xlabel=feature,
        ylabel=f'% {target}',
        title=f"{feature}: prob of '{target}'"
    )
    # Plot a horizontal line at the baseline prediction
    # probability: two classes -> prob=0.5
    ax3.axhline(
        y=0.5, color='green', alpha=0.4, linestyle='--'
    )

# Data Preprocessing ⚙️🧹

In [3]:
# 'PassengerId' 컬럼 드랍 (상관관계 X)
# 제출할 때는 컬럼 다시 추가해줘야 합니다.
def drop_PassengerId(df_train):
    df_train = df_train.drop(columns=['PassengerId'])
    return df_train

In [4]:
# 'Name' 컬럼에서 'honorific_title' 컬럼 생성하기
# 의미있는 호칭(Mr, Mrs, Miss, Master)을 추출하는 과정입니다.

# 'honorific_title' 생성하는 함수
def get_honorific_title(name):
    top4_titles = ('Mr', 'Mrs', 'Miss', 'Master')

    # top4 호칭으로 바꿔줄 소수 여성 호칭
    female_titles_to_Mrs = ('Mme', 'the Countess', 'Dona', 'Lady')
    female_titles_to_Miss = ('Mlle', 'Ms')

    # top4 호칭으로 바꿔줄 소수 남성 호칭
    male_titles_to_Mr = ('Major', 'Col', 'Capt', 'Don', 'Sir', 'Jonkheer', 'Rev')

    # 예 : "Braund, Mr. Owen Harris"에서 'Mr' 추출
    # '.'(마침표) 기준으로 이름을 자르고, ','(콤마) 제거
    honorific_title = name.split('.')[0].split(', ')[-1]

    # 호칭에 따라 변환해서 리턴
    if honorific_title in top4_titles:
        return honorific_title
    elif honorific_title in female_titles_to_Mrs:
        return 'Mrs'
    elif honorific_title in female_titles_to_Miss:
        return 'Miss'
    elif honorific_title in male_titles_to_Mr:
        return 'Mr'
    else:
        return np.nan

In [5]:
# 새 피쳐로 사용할 컬럼 'honorific_title' 생성
def make_honorific_title(df_train, df_test):
    df_train['honorific_title'] = df_train['Name'].map(get_honorific_title)
    df_test['honorific_title'] = df_test['Name'].map(get_honorific_title)

    return df_train, df_test

In [6]:
# 원래 컬럼 'Name'를 드랍
def drop_Name(df_train, df_test):
    df_train = df_train.drop(columns=['Name'])
    df_test = df_test.drop(columns=['Name'])

    return df_train, df_test

In [7]:
# 새 컬럼 'Family' 생성 후 레이블 인코딩
def make_Family_column(df_train, df_test):
    # 'Famliy' 컬럼 계산해서 생성
    df_train['Family'] = df_train['SibSp'].astype(int) + df_train['Parch'].astype(int) + 1
    df_test['Family'] = df_test['SibSp'].astype(int) + df_test['Parch'].astype(int) + 1

    # 'Famliy' 컬럼 레이블 인코딩
    df_train['Family'] = df_train['Family'].map(
        lambda f : str(f) if f < 3 else '>=3'
    )

    df_test['Family'] = df_test['Family'].map(
        lambda f : str(f) if f < 3 else '>=3'
    )

    return df_train, df_test

In [8]:
# 'SibSp' 컬럼 드랍
def drop_SibSp(df_train, df_test):
    df_train = df_train.drop(columns=['SibSp'])
    df_test = df_test.drop(columns=['SibSp'])

    return df_train, df_test

In [9]:
# 'Parch' 컬럼 드랍
def drop_Parch(df_train, df_test):
    df_train = df_train.drop(columns=['Parch'])
    df_test = df_test.drop(columns=['Parch'])

    return df_train, df_test

In [10]:
# 'Cabin' 컬럼 U로 채우고 매핑
def mapping_Cabin(df_train, df_test):
    # Cabin 빈칸 채우기
    df_train['Cabin'] = df_train['Cabin'].fillna('Unknown')
    df_test['Cabin'] = df_test['Cabin'].fillna('Unknown')

    # Cabin의 머리글자를 추출하고 매핑하기 위한 함수 생성
    cabin_mapping = {
        cabin[0]: idx for idx, cabin in enumerate(sorted(set(cabin[0] for cabin in df_train['Cabin'].unique())))
    }

    # 매핑 적용
    df_train['Cabin'] = df_train['Cabin'].map(lambda x : cabin_mapping[x[0]])
    df_test['Cabin'] = df_test['Cabin'].map(lambda x : cabin_mapping[x[0]])

    return df_train, df_test

In [11]:
# 'Ticket' 컬럼 드랍 (의미를 찾을수 없음)
def drop_Ticket(df_train, df_test):
    df_train = df_train.drop(columns=['Ticket'])
    df_test = df_test.drop(columns=['Ticket'])

    return df_train, df_test

In [12]:
# 'Age' 결측치 채우는 클래스
class AgeImputer(
    SimpleImputer, TransformerMixin
):
    """
    Age Imputer from 'honorific_title' column.

    There are four honorific titles: Mr, Mrs, Miss, Master.
    We proved that these groups have statistically different
    mean Ages (Independent T-tests).

    Imputation strategy:
        - fit:  compute the mean Age for each honorifit title
            Compute the mean Age for those who do not have title
        - transform:  if Age is missing for a passenger, if it has
            honorific title, the mean Age for that honorific title
            will be assigned. Else, the mean age for those who do
            not have title will be assigned.
    """

    def __init__(self) -> None:
        """
        Initialize an AgeImputer instance
        """

        # Initialize the parent Sklearn classes
        super().__init__()
        # Define the mappig {title_: mean Age for title_}
        self.title2age = {}
        # Define the mean age of those people with no title
        self.no_title_mean_age = None
        self.is_fitted = False


    def fit(self, X: pd.DataFrame, y=None):
        """
        Fits the 'title2age' and 'honorific_title' using 'X'.

        'X' is assumed to be Training Data (not Test Data!)
        'X' is assumed to be a non-empty data frame with
            (at least) the columns 'Age' and 'honorific_title'
        """

        df_groupby = df_train.groupby(
            by='honorific_title'
        ).agg({'Age': 'mean'})
        self.title2age = df_groupby.to_dict()['Age']

        if X['honorific_title'].isna().sum() == 0:
            self.no_title_mean_age = X['Age'].mean()
        else:
            self.no_title_mean_age = X.loc[
                X['honorific_title'].isna(), 'Age'
            ].mean()
        self.is_fitted = True
        return self


    def transform(self, X: pd.DataFrame, y=None) -> pd.DataFrame:
        """
        Transform 'X' by imputing the missing values of 'Age'

        'X' is assumed to be Training Data (not Test Data!)
        'X' is assumed to be a non-empty data frame with
            (at least) the columns 'Age' and 'honorific_title'
        """

        if not self.is_fitted:
            raise Exception("Call the 'fit' method first")

        X_out = X.copy(deep=True)
        missing_age_idx = X.loc[X['Age'].isna(), :].index
        age_col_idx = np.where(X.columns == 'Age')[0][0]

        for idx in missing_age_idx:

            # get the honorific_title of the 'idx' row
            title = X.loc[idx,'honorific_title']

            if title in self.title2age:
                X_out.loc[idx, 'Age'] = self.title2age[title]
            else:
                X_out.loc[idx, 'Age'] = self.no_title_mean_age

        return X_out['Age'].to_frame()

In [13]:
# Create three imputers : 결측치 채우는 함수를 imputer라고 합니다.
age_imputer = AgeImputer().set_output(transform='pandas')

fare_imputer = SimpleImputer(
    strategy='median'
).set_output(transform='pandas')

embarked_imputer = SimpleImputer(
    strategy='most_frequent'
).set_output(transform='pandas')


# Create a general imputer (for all the variables)
imputer = ColumnTransformer(
    transformers=[
        ('age_imp', age_imputer, ['Age', 'honorific_title']),
        ('fare_imp', fare_imputer, ['Fare']),
        ('embarked_imp', embarked_imputer, ['Embarked'])
    ],
    remainder='passthrough', # 나머지는 놔둔다.
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [14]:
# 'honorific_title' 한정 원핫 인코딩(나이 결측치 채우는 데 필요)
def make_honorific_title_dummy(df_train, df_test):
    df_train['dummy_honorific_title'] = df_train['honorific_title']
    df_train = pd.get_dummies(
          data=df_train
        , columns=['dummy_honorific_title']
        , prefix=''
        , prefix_sep=''
        , dummy_na=False  # Null 컬럼은 복사하지 않는다.
        , drop_first=False  # Top4 honorific titles 행은 전부 복사
        , dtype=int
    )

    df_test['dummy_honorific_title'] = df_test['honorific_title']
    df_test = pd.get_dummies(
          data=df_test
        , columns=['dummy_honorific_title']
        , prefix=''
        , prefix_sep=''
        , dummy_na=False  # Null 컬럼은 복사하지 않는다.
        , drop_first=False  # Top4 honorific titles 행은 전부 복사
        , dtype=int
    )

    return df_train, df_test

In [15]:
# 원핫 인코딩
def one_hot_encoder(df_train, df_test):
  df_train = pd.get_dummies(
        data=df_train
      , dummy_na=False
      , drop_first=False
      , dtype=int
  ).drop(columns='honorific_title')

  df_test = pd.get_dummies(
        data=df_test
      , dummy_na=False
      , drop_first=False
      , dtype=int
  ).drop(columns='honorific_title')

  return df_train, df_test

In [16]:
# 컬럼 생성 및 드랍하는 함수 모음
def make_drop_columns(df_train, df_test):
    # 바로 드랍하는 컬럼들 드랍
    df_train = drop_PassengerId(df_train)
    df_train, df_test = drop_Ticket(df_train, df_test)

    # 전처리 후 인코딩하는 컬럼
    df_train, df_test = mapping_Cabin(df_train, df_test)
    df_train, df_test = make_honorific_title(df_train, df_test)
    df_train, df_test = drop_Name(df_train, df_test)

    # 'Family' 컬럼 만들고 필요없는 컬럼 드랍
    df_train, df_test = make_Family_column(df_train, df_test)
    df_train, df_test = drop_SibSp(df_train, df_test)
    df_train, df_test = drop_Parch(df_train, df_test)

    # honorific_title의 더미 컬럼 생성
    df_train, df_test = make_honorific_title_dummy(df_train, df_test)

    return df_train, df_test

In [17]:
# 컬럼 생성/드랍 함수 실행
df_train, df_test = make_drop_columns(df_train, df_test)

In [18]:
# 결측치 채우기
df_train = pd.DataFrame(
      imputer.fit_transform(df_train)
    , columns=df_train.columns
)

df_test = pd.DataFrame(
      imputer.fit_transform(df_test)
    , columns=df_test.columns
)

In [19]:
# 원핫 인코딩
df_train, df_test = one_hot_encoder(df_train, df_test)

# RandomForestClassifier Training 🏋️‍♂️🌲

In [20]:
# 학습데이터의 데이터와 답 분리
X_train = df_train.drop(columns=['Survived'])
y_train = df_train['Survived']

# 테스트데이터 생성
X_test = df_test.drop(columns='PassengerId')

In [34]:
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 주입
params = {
      'max_depth' : [10,20,30]
    , 'min_samples_leaf' : [1,6,12]
    , 'min_samples_split' : [2,8,16]
}
# RF 모델 생성
rf_clf = RandomForestClassifier(
      n_estimators=100
    , random_state=42
    , n_jobs=-1
)
# GridSearchCV 학습
grid_cv = GridSearchCV(
  rf_clf
  , param_grid=params
  , cv=2
  , n_jobs=-1
)
grid_cv.fit(X_train, y_train)

GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_leaf': [1, 6, 12],
                         'min_samples_split': [2, 8, 16]})

In [35]:
grid_cv.best_params_

{'max_depth': 10, 'min_samples_leaf': 12, 'min_samples_split': 2}

In [36]:
grid_cv.best_score_

0.802453771350834

# Submission File 🏆🎯

In [37]:
# 위에서 찾은 최적의 hyperparameters set을 이용한 최고의 모델 생성
rf_clf = RandomForestClassifier(
      n_estimators=100
    , min_samples_leaf=12
    , max_depth=10
    , min_samples_split=2
    , random_state=42
)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=12, random_state=42)

In [38]:
# 테스트데이터 넣어서 예측
pred = rf_clf.predict(X_test)

In [39]:
# 제출파일 준비: PassengerId - Survived
submission = pd.DataFrame({
    'PassengerId' : df_test['PassengerId'],
    'Survived' : pred
})

submission['Survived'] = submission['Survived'].astype(int)

In [40]:
# 제출파일 저장
submission.to_csv('submission.csv', index=False)
print('your submission file is saved!')

your submission file is saved!


## 채점하기

In [41]:
# 생성된 데이터 파일과 솔루션 불러오기
# 파일 경로 적절히 수정해서 쓰세요.
submission = pd.read_csv('./submission.csv')
solution = pd.read_csv('./solution.csv')

In [42]:
# 채점 테이블 생성
marking = submission.merge(solution, on='PassengerId')
marking.columns = ['PassengerId', 'Submission', 'Solution']

# 'Check' : 맞으면 1, 틀리면 0
marking['Check'] = 0

In [43]:
# 채점
cond = marking['Submission'] == marking['Solution']
marking.loc[cond, 'Check'] = 1

# 정답 계산 및 출력
result = marking['Check'].sum() / marking.shape[0]
print('정답: {0:.5f}'.format(result))

정답: 0.78947
